### Leitura dos Arquivos

In [27]:
import pandas as pd
import os

def ler_arquivo(nome_df, path, col_codigo_empresa = 'Codigo Empresa', col_data_ref = 'Data Ref',col_valor_atual = 'Valor Trimestre Atual',col_cod_conta = 'Codigo Conta', 
                col_desc_conta = 'Descricao Conta'):
    df_transform = pd.read_csv(path, sep=";")
    df_transform['Codigo_Descricao_Conta'] = df_transform[col_cod_conta].astype(str) + ' - ' + df_transform[col_desc_conta]
    df_conta = df_transform[['Codigo Conta', 'Descricao Conta', 'Codigo_Descricao_Conta']].copy()
    df_conta['nome_df'] = nome_df
    df_transform.drop(columns=[col_cod_conta, col_desc_conta], inplace=True)
    df_transform = df_transform.pivot_table(index=[col_codigo_empresa, col_data_ref], columns='Codigo_Descricao_Conta', values= col_valor_atual, aggfunc='sum')
    df_transform = df_transform.rename_axis(columns=None)
    df_transform.reset_index(inplace=True)
    df_transform[col_data_ref] = pd.to_datetime(df_transform[col_data_ref], format='%Y%m%d')
    df_transform.sort_values(by=col_data_ref, inplace=True, ascending=False)
    
    return(df_transform, df_conta) # df transformado e um df_conta para ajudar a encontrar as contas sem precisar ir nos arquivos

csv_folder = 'Data\\02_Silver'
run_date = '20240601_155051'

df_cons_ativo, aux_conta_cons_ativo = ler_arquivo(nome_df = 'df_cons_ativo', path = f"{os.path.join(csv_folder, run_date)}_DF Cons Ativo.csv")

df_cons_passivo, aux_conta_cons_passivo = ler_arquivo(nome_df = 'df_cons_passivo', path = f"{os.path.join(csv_folder, run_date)}_DF Cons Passivo.csv")

df_cons_resultado_per, aux_conta_resultado_per = ler_arquivo(nome_df = 'df_cons_resultado_per', path = f"{os.path.join(csv_folder, run_date)}_DF Cons Resultado Periodo.csv")

df_ind_passivo, aux_conta_ind_passivo = ler_arquivo(nome_df = 'df_ind_passivo', path = f"{os.path.join(csv_folder, run_date)}_DF Ind Passivo.csv")

df_cons_fluxo_caixa, aux_conta_cons_fluxo_caixa = ler_arquivo(nome_df = 'df_cons_fluxo_caixa', path = f"{os.path.join(csv_folder, run_date)}_DF Cons Fluxo de Caixa.csv")

# Concatenar todos os DataFrames aux_conta
aux_conta = pd.concat([aux_conta_cons_ativo, aux_conta_cons_passivo, aux_conta_resultado_per, aux_conta_ind_passivo]).drop_duplicates()

In [26]:
coluna = 'Descricao Conta'
texto = 'Amortização'
aux_conta[aux_conta[coluna].str.contains(texto, case=False, na=False)]

,Codigo Conta,Descricao Conta,Codigo_Descricao_Conta,nome_df


### Criação dos indicadores

In [30]:
def merged_dfs(df1, df2, on, columns):
    merged_df = pd.merge(df1, df2, on=on)[columns]
    for col in columns:
        if merged_df[col].dtype == 'object':
            merged_df[col] = merged_df[col].str.replace('.', '').astype(float)
    return merged_df

def unique_dfs(df1, on, columns):
    unique_df = df1[columns].copy()
    for col in columns:
        if unique_df[col].dtype == 'object':
            unique_df[col] = unique_df[col].str.replace('.', '').astype(float)
    return unique_df

# Iniciando um df só com a data e código da empesa
df_indicadores = df_cons_ativo[["Codigo Empresa", "Data Ref"]]

# LiquidezCorrente
indicador = ['LiquidezCorrente']
on = ['Codigo Empresa', 'Data Ref']
columns_cal = ['1.01 - Ativo Circulante', '2.01 - Passivo Circulante']

columns = on + columns_cal
merged_df = merged_dfs(df_cons_ativo, df_cons_passivo, on, columns)

merged_df[indicador[0]] = merged_df['1.01 - Ativo Circulante']/merged_df['2.01 - Passivo Circulante'] # Cálculo

columns_keep = on + indicador
merged_df = merged_df[columns_keep]

df_indicadores = pd.merge(df_indicadores, merged_df, on=['Codigo Empresa', 'Data Ref'])
df_indicadores


# Passivos_vs_Ativos"
indicador = ['Passivos_vs_Ativos']
on = ['Codigo Empresa', 'Data Ref']
columns_cal = ['2.01 - Passivo Circulante', '2.02 - Passivo Não Circulante', '1 - Ativo Total']

columns = on + columns_cal
merged_df = merged_dfs(df_cons_ativo, df_cons_passivo, on, columns)

merged_df[indicador[0]] = (merged_df['2.01 - Passivo Circulante'] + merged_df['2.02 - Passivo Não Circulante'])/merged_df['1 - Ativo Total'] # Cálculo

columns_keep = on + indicador
merged_df = merged_df[columns_keep]

df_indicadores = pd.merge(df_indicadores, merged_df, on=['Codigo Empresa', 'Data Ref'])
df_indicadores

# PL
indicador = ['PL']
on = ['Codigo Empresa', 'Data Ref']
columns_cal = ['2.03 - Patrimônio Líquido']

columns = on + columns_cal
merged_df = unique_dfs(df_ind_passivo, on, columns)

merged_df[indicador[0]] = merged_df['2.03 - Patrimônio Líquido']

columns_keep = on + indicador
merged_df = merged_df[columns_keep]

df_indicadores = pd.merge(df_indicadores, merged_df, on=['Codigo Empresa', 'Data Ref'])
df_indicadores


# PL_vs_Ativos
indicador = ['PL_vs_Ativos']
on = ['Codigo Empresa', 'Data Ref']
columns_cal = ['2.03 - Patrimônio Líquido', '1 - Ativo Total']

columns = on + columns_cal
merged_df = merged_dfs(df_ind_passivo, df_cons_ativo, on, columns)
merged_df[indicador[0]] = merged_df['2.03 - Patrimônio Líquido']/merged_df['1 - Ativo Total']

columns_keep = on + indicador
merged_df = merged_df[columns_keep]

df_indicadores = pd.merge(df_indicadores, merged_df, on=['Codigo Empresa', 'Data Ref'])
df_indicadores

# DividaBruta
indicador = ['DividaBruta']
on = ['Codigo Empresa', 'Data Ref']
columns_cal = ['2.01.04 - Empréstimos e Financiamentos', '2.02.01 - Empréstimos e Financiamentos']

columns = on + columns_cal
merged_df = unique_dfs(df_cons_passivo, on, columns)

merged_df[indicador[0]] = merged_df['2.01.04 - Empréstimos e Financiamentos'] + merged_df['2.02.01 - Empréstimos e Financiamentos']

columns_keep = on + indicador
merged_df = merged_df[columns_keep]

df_indicadores = pd.merge(df_indicadores, merged_df, on=['Codigo Empresa', 'Data Ref'])
df_indicadores

# DividaLiquida
indicador = ['DividaLiquida']
on = ['Codigo Empresa', 'Data Ref']
columns_cal = ['2.01.04 - Empréstimos e Financiamentos', '2.02.01 - Empréstimos e Financiamentos', '1.01.01 - Caixa e Equivalentes de Caixa', '1.01.02 - Aplicações Financeiras']

columns = on + columns_cal
merged_df = merged_dfs(df_cons_passivo, df_cons_ativo, on, columns)

merged_df[indicador[0]] = (merged_df['2.01.04 - Empréstimos e Financiamentos'] + merged_df['2.02.01 - Empréstimos e Financiamentos']) - \
                            (merged_df['1.01.01 - Caixa e Equivalentes de Caixa'] + merged_df['1.01.02 - Aplicações Financeiras'])

columns_keep = on + indicador
merged_df = merged_df[columns_keep]

df_indicadores = pd.merge(df_indicadores, merged_df, on=['Codigo Empresa', 'Data Ref'])
df_indicadores

# DividaLiquida_vs_PL
indicador = ['DividaLiquida_vs_PL']
df_indicadores[indicador[0]] = df_indicadores['DividaLiquida']/df_indicadores['PL']
df_indicadores

# EBIT
indicador = ['EBIT']
on = ['Codigo Empresa', 'Data Ref']
columns_cal = ['3.04 - Despesas/Receitas Operacionais', '3.03 - Resultado Bruto']

columns = on + columns_cal
merged_df = unique_dfs(df_cons_resultado_per, on, columns)

merged_df[indicador[0]] = merged_df['3.04 - Despesas/Receitas Operacionais'] + merged_df['3.03 - Resultado Bruto']

columns_keep = on + indicador
merged_df = merged_df[columns_keep]

df_indicadores = pd.merge(df_indicadores, merged_df, on=['Codigo Empresa', 'Data Ref'])
df_indicadores

# EBITDA
indicador = ['EBITDA']
on = ['Codigo Empresa', 'Data Ref']
columns_cal = ['3.04 - Despesas/Receitas Operacionais', '3.03 - Resultado Bruto', '6.01.01.06 - Depreciação, Amortização e exaustão']

columns = on + columns_cal
merged_df = merged_dfs(df_cons_fluxo_caixa, df_cons_resultado_per, on, columns)

merged_df[indicador[0]] = merged_df['3.04 - Despesas/Receitas Operacionais'] + merged_df['3.03 - Resultado Bruto'] + merged_df['6.01.01.06 - Depreciação, Amortização e exaustão']

columns_keep = on + indicador
merged_df = merged_df[columns_keep]

df_indicadores = pd.merge(df_indicadores, merged_df, on=['Codigo Empresa', 'Data Ref'])
df_indicadores

# DividaLiquida_vs_PL
indicador = 'DividaLiquida_vs_PL'
df_indicadores[indicador] = df_indicadores['DividaLiquida']/df_indicadores['PL']
df_indicadores

,Codigo Empresa,Data Ref,LiquidezCorrente,Passivos_vs_Ativos,PL,PL_vs_Ativos,DividaBruta,DividaLiquida,DividaLiquida_vs_PL,EBIT,EBITDA
0,7617,2024-03-31,2.706354,0.187061,80435000.0,0.773666,12189000.0,3803000.0,0.047280,3756000.0,4059000.0
1,7617,2023-09-30,1.647388,0.184701,79738000.0,0.775548,11032000.0,2824000.0,0.035416,11086000.0,11889000.0
2,7617,2023-06-30,1.494513,0.211235,76867000.0,0.751190,12606000.0,5000000.0,0.065047,7464000.0,8018000.0
3,7617,2023-03-31,1.460624,0.209001,73632000.0,0.752307,12237000.0,5956000.0,0.080889,3323000.0,3582000.0
4,7617,2021-09-30,1.592067,0.180318,62602000.0,0.772797,8186000.0,4257000.0,0.068001,9582000.0,10123000.0
